In [36]:
import sqlite3
import pandas as pd
import os
import copy
pd.options.mode.chained_assignment = None

In [37]:
process_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","02_processed","intermediate.db")
print(process_path)

/home/jupyter/Team-Prophecy/Data/02_processed/intermediate.db


In [38]:
process_connection = sqlite3.connect(process_path)

In [39]:
inputs = {    
    'prog_desc'     : "All",
    'courses'       : "All",
    'mod'           : "F2F",
    'visa'          : "F1 Visa",
    'ExpN_eat'      : 150
}

In [40]:
# Building the model parameters
model_params = {
    'prog_desc'         : inputs['prog_desc'],
    'courses'           : inputs['courses'],
    'mod'               : inputs['mod'],
    'visa'              : inputs['visa'],
    'ExpN_eat'          : inputs['ExpN_eat'],
    'enr_hist'          : 4 if inputs['visa'] == 'F1 Visa' else 8,
    'filters'           : {
        'prog_filter'   : inputs['prog_desc'] != 'All',
        'course_filter' : inputs['courses'] != 'All',
        'visa_filter'   : inputs['visa'] != 'combined'
    },
    'groupby'           : {
        'mod_groupby'   : inputs['mod'] != 'combined'
    }
}

In [41]:
def list_to_str(l):
    """
    Create a string in the format of "('a', 'b', 'c', 'd')" from list ['a', 'b', 'c', 'd']
    """
    return str(tuple(l))

In [42]:
student_stat_query = """
    SELECT rs.reg_term_code, rs.reg_stu_id, crs, rs.sect_id, rs.reg_new_ret_stu, rs.reg_final_status, scoff.cum_seat_total, scoff.cum_seat_enroll, scoff.cum_seat_avail
    FROM registration_status rs
    INNER JOIN (SELECT sc.cum_term, i_n.instr_home_org, sc.cum_instr, sc.cum_term_code, sc.cum_sect_id, 
        sc.cum_seat_enroll+abs(sc.cum_seat_avail)+sc.cum_seat_wait as cum_seat_total,
        IIF(sc.cum_seat_avail < 0, sc.cum_seat_enroll+abs(sc.cum_seat_avail),sc.cum_seat_enroll) as cum_seat_enroll,
        IIF(sc.cum_seat_avail < 0, 0, sc.cum_seat_avail) as cum_seat_avail
    FROM semester_course_offerings sc
    INNER JOIN instructors i_n ON sc.cum_instr == i_n.instr_name) scoff ON 
        scoff.cum_term == rs.reg_term_code
        AND scoff.cum_term_code == rs.crs
        AND scoff.cum_sect_id == rs.sect_id
    WHERE 1=1 
"""
student_details_query = """
    SELECT stu_admit_term_code, stu_college, stu_deg_level, stu_dept, stu_id, stu_res, stu_prog, stu_visa, stu_bam
    FROM student_details 
    WHERE 1=1 
"""

program_course_offerings = """
    SELECT * FROM PROGRAM_COURSE_OFFERINGS 
"""

# Program Filter
if model_params['filters']['prog_filter']:
    student_details_query += f"AND stu_prog = '{model_params['prog_desc']}' "

# Course Filter
if model_params['filters']['course_filter']:
    student_stat_query += f"AND crs IN {list_to_str(model_params['courses'])} "

# Visa Filter
if model_params['filters']['visa_filter']:
    student_details_query += f"AND stu_visa = '{model_params['visa']}' "

In [43]:
student_stat = pd.DataFrame(process_connection.execute(student_stat_query).fetchall(),columns=["reg_term_code", "student_id", "crs", "sect_id", "returning_student", "reg_status",
                                                                                              "cum_total_enrollment","cum_seat_enrollment","cum_seat_available"])

#WHERE reg_final_status IN ('W','R')
#GROUP BY reg_term_code, crs, sect_id, reg_final_status

student_details = pd.DataFrame(process_connection.execute(student_details_query).fetchall(),columns=["reg_term_code","stu_college","stu_deg_level","stu_dept","student_id",
                                                                                                     "stu_res","stu_prog","stu_visa","stu_bam"])

pco_df = pd.DataFrame(process_connection.execute(program_course_offerings),columns=["stu_prog","crs","required"])

total_stat = student_stat.merge(student_details, on=["reg_term_code","student_id"], how="inner").fillna(0)
total_stat = total_stat.merge(pco_df, on=["stu_prog","crs"])

In [44]:
total_stat = total_stat.loc[total_stat["required"] == 1,:]

In [45]:
total_student_population = total_stat[["reg_term_code"]].groupby("reg_term_code").count().reset_index()

In [46]:
reg_term_values = dict(total_stat.groupby(["reg_term_code"])["reg_term_code"].count())

In [47]:
program_limit_values = dict(total_stat.groupby(["stu_prog"])["stu_prog"].count())

In [48]:
total_courses = total_stat[["reg_term_code","stu_prog","crs","sect_id","reg_status","cum_total_enrollment","cum_seat_enrollment","cum_seat_available"]]

REF Courses is just a fancy way of determining how, from a professors point of view, courses are included.

In [49]:
ref_course = total_courses[["reg_term_code","crs","sect_id","cum_total_enrollment","cum_seat_enrollment","cum_seat_available"]]

In [50]:
registered_courses = total_courses.loc[total_courses["reg_status"] == "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
#nonregistered_courses = total_courses.loc[total_courses["reg_status"] != "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

Keep in mind: Our registered values will be dependent on what's in our subset.

In [51]:
req_courses_context = registered_courses.merge(ref_course[["reg_term_code", "crs","sect_id", "cum_total_enrollment"]], on=["reg_term_code", "crs","sect_id"], how="inner").drop_duplicates()

In [52]:
req_courses_context.columns

Index(['reg_term_code', 'stu_prog', 'crs', 'sect_id', 'reg_status',
       'cum_total_enrollment'],
      dtype='object')

In [19]:
req_courses_context.loc[req_courses_context["cum_total_enrollment"] < req_courses_context["reg_status"],["cum_total_enrollment"]]  = req_courses_context["reg_status"]

In [53]:
reg_term_courses_context = req_courses_context.groupby(["reg_term_code"])

In [54]:
req_courses_context.head()

,reg_term_code,stu_prog,crs,sect_id,reg_status,cum_total_enrollment
0,202010,EC-MS-AIT,AIT524,003,2,25
2,202010,EC-MS-AIT,AIT542,002,2,20
4,202010,EC-MS-AIT,AIT602,001,1,25
5,202010,EC-MS-AIT,AIT673,DL1,1,25
6,202010,EC-MS-AIT,AIT677,001,3,25


In [55]:
from ortools.linear_solver import pywraplp
from itertools import chain

For each semester's influx of students, we want the following:
- How many butts are going to be in a seat for a given course section
- How many butts are going to be waitlisted for a given course section

**DECISION VARIABLES**

X = Number of Students who applied

W = Whether the student was waitlisted for the class section

A = Whether the student has already registered for the class

B = Whether the student has already waitlisted for the class

In [56]:
all_semesters = list(reg_term_values.keys())
#program_limit_values <- included because we need a way to constraint students
all_pcs_list = list(req_courses_context[["stu_prog","crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
all_pcs_dict = {}
#References what program contains which class
pc_dict = {}
cs_dict = {}
for t_pcs in all_pcs_list:
    if t_pcs[0] not in all_pcs_dict:
        pc_dict[t_pcs[0]] = []
        all_pcs_dict[t_pcs[0]] = {}
    pc_dict[t_pcs[0]].append(t_pcs[1])
    if t_pcs[1] not in all_pcs_dict[t_pcs[0]]:
        cs_dict[t_pcs[1]] = []
        all_pcs_dict[t_pcs[0]][t_pcs[1]] = []
    cs_dict[t_pcs[1]].append(t_pcs[2])
    all_pcs_dict[t_pcs[0]][t_pcs[1]].append(t_pcs[2])

In [57]:
req_courses_context
#req_courses_context[["reg_term_code","stu_prog","crs","sect_id","reg_status"]]

,reg_term_code,stu_prog,crs,sect_id,reg_status,cum_total_enrollment
0,202010,EC-MS-AIT,AIT524,003,2,25
2,202010,EC-MS-AIT,AIT542,002,2,20
4,202010,EC-MS-AIT,AIT602,001,1,25
5,202010,EC-MS-AIT,AIT673,DL1,1,25
6,202010,EC-MS-AIT,AIT677,001,3,25
...,...,...,...,...,...,...
1544,202310,EC-MS-CEIE,CEIE767,001,2,15
1546,202310,EC-MS-CEIE,CEIE795,001,55,60
1601,202310,EC-MS-DAEN,INFS580,002,2,4
1603,202310,EC-MS-DAEN,INFS580,003,1,12


In [58]:
req_courses_context["stu_prog"].drop_duplicates().tolist()

['EC-MS-AIT', 'EC-MS-CEIE', 'EC-MS-DAEN', 'EC-MS-ISYS']

In [59]:
#Constraints
#total_student_population
#program_limit_values

Fall 2020
150 -> 80 Program_1, 40 Program_2, 30 Program_3
50
1 student -> 15 credits = 3-4 classes

In [111]:
for a_sem in all_semesters:
    cxt_grp = reg_term_courses_context.get_group(a_sem)
    #WE HAVE RATIOS HERE IN CASE WE NEED TO DO MORE ANALYSIS OR MATCH IT TO 1
    #
    registered_list = cxt_grp["reg_status"]/reg_term_values[a_sem]
    #total_students_list = cxt_grp["cum_total_enrollment"]/reg_term_values[a_sem]
    pcs_weighting_ratio = cxt_grp[["reg_term_code","stu_prog","crs","sect_id","reg_status"]].set_index(["reg_term_code","stu_prog","crs","sect_id"])["reg_status"].to_dict()
    pcs_weighting_ratio = {"("+",".join(k)+")" : v for k,v in pcs_weighting_ratio.items()}
    #Now that we have the ratios, we can begin with the objective function.
    #
    decision_vars = {}
    const_vars = {}
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Cannot get solver")
        continue
        
        
    #These will be our constraints
    total_capacity = cxt_grp[["crs","reg_status"]].set_index("crs").to_dict()["reg_status"]
    
    program_l = cxt_grp["stu_prog"].drop_duplicates().tolist()
    cs_l = cxt_grp["crs"].drop_duplicates().tolist()

    #These will be our reference variables
    program_ref = {}
    course_ref = {}
    section_ref = {}
    
    program_w_ref = {}
    course_w_ref = {}
    course_w_ref = {}
    section_w_ref = {}
    
    course_a_ref = {}
    section_a_ref = {}
    
    course_b_ref = {}
    section_b_ref = {}
    
    #Student ID
    x_decision_vars = {}
    w_decision_vars = {}
    a_decision_vars = {}
    b_decision_vars = {}
    #
    s_inf = solver.infinity()
    
    all_pcs = list(cxt_grp[["stu_prog","crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
    for a_pc in all_pcs:
        for student_id in range(1,reg_term_values[a_sem]):
            #We'll have two primary decision variables: Waitlist and Student
            #This will be a minimization function for our purposes, as we want all students
            # to be included.

            a_name = f"a({a_pc[0]},{a_pc[1]},{a_pc[2]},{student_id},{a_sem})"
            b_name = f"b({a_pc[0]},{a_pc[1]},{a_pc[2]},{student_id},{a_sem})"
            
            x_name = f"x({a_pc[0]},{a_pc[1]},{a_pc[2]},{student_id},{a_sem})"
            w_name = f"w({a_pc[0]},{a_pc[1]},{a_pc[2]},{student_id},{a_sem})"
            
            #Declares the decision variable
            decision_vars[a_name] = solver.IntVar(0,1,a_name)
            decision_vars[b_name] = solver.IntVar(0,1,b_name)
            decision_vars[x_name] = solver.IntVar(0,s_inf,x_name)
            decision_vars[w_name] = solver.IntVar(0,s_inf,w_name)
            
            #waitlist_decision_vars[student_id] = pcs_weighting_ratio[f"({a_sem},{a_pc[0]},{a_pc[1]},{a_pc[2]})"]
            x_decision_vars[student_id] = decision_vars[x_name]
            w_decision_vars[student_id] = decision_vars[w_name]
            a_decision_vars[student_id] = decision_vars[a_name]
            b_decision_vars[student_id] = decision_vars[b_name]
            
            if a_pc[0] not in program_ref:
                program_ref[a_pc[0]] = {}
                program_w_ref[a_pc[0]] = {}
            if student_id not in program_ref[a_pc[0]]:
                program_ref[a_pc[0]][student_id] = []
                program_w_ref[a_pc[0]][student_id] = []
            program_ref[a_pc[0]][student_id].append(decision_vars[x_name])
            program_w_ref[a_pc[0]][student_id].append(decision_vars[w_name])
            
            if a_pc[1] not in course_ref:
                course_ref[a_pc[1]] = {}
                course_w_ref[a_pc[1]] = {}
                course_a_ref[a_pc[1]] = {}
                course_b_ref[a_pc[1]] = {}
            if student_id not in course_ref[a_pc[1]]:
                course_ref[a_pc[1]][student_id] = []
                course_w_ref[a_pc[1]][student_id] = []
                course_a_ref[a_pc[1]][student_id] = []
                course_b_ref[a_pc[1]][student_id] = []
            course_ref[a_pc[1]][student_id].append(decision_vars[x_name])
            course_w_ref[a_pc[1]][student_id].append(decision_vars[w_name])
            course_a_ref[a_pc[1]][student_id].append(decision_vars[a_name])
            course_b_ref[a_pc[1]][student_id].append(decision_vars[b_name])
            
            if a_pc[1] not in section_ref:
                section_ref[a_pc[1]] = {}
                section_w_ref[a_pc[1]] = {}
                section_a_ref[a_pc[1]] = {}
                section_b_ref[a_pc[1]] = {}
            if a_pc[2] not in section_ref[a_pc[1]]:
                section_ref[a_pc[1]][a_pc[2]] = []
                section_w_ref[a_pc[1]][a_pc[2]] = []
                section_a_ref[a_pc[1]][a_pc[2]] = []
                section_b_ref[a_pc[1]][a_pc[2]] = []
            section_ref[a_pc[1]][a_pc[2]].append(decision_vars[x_name])
            section_w_ref[a_pc[1]][a_pc[2]].append(decision_vars[w_name])
            section_a_ref[a_pc[1]][a_pc[2]].append(decision_vars[a_name])
            section_b_ref[a_pc[1]][a_pc[2]].append(decision_vars[b_name])
    
    
    for p in program_l:
        const_vars[f"01_program_{p}_constr"] = solver.Add(solver.Sum([solver.Sum(program_ref[p][s]) + solver.Sum(program_w_ref[p][s]) 
                                                           for s in range(1,reg_term_values[a_sem])]) 
                                                          >= program_limit_values[p],name=f"01_program_{p}_constr")
    
    for student_id in range(1,reg_term_values[a_sem]):
        for cs in cs_l:
            const_vars[f"02_course_{cs}_constr"] = solver.Add(solver.Sum(course_ref[cs][student_id]) + solver.Sum(course_w_ref[cs][student_id]) >= total_capacity[cs], name=f"02_course_with_w_{cs}_constr")
            
            const_vars[f"02_course_{cs}_constr"] = solver.Add(solver.Sum(course_ref[cs][student_id]) <= total_capacity[cs]*(solver.Sum(course_a_ref[cs][student_id])),name=f"02_course_{cs}_constr")
            const_vars[f"02_course_{cs}_constr"] = solver.Add(solver.Sum(course_w_ref[cs][student_id]) <= 100*(solver.Sum(course_b_ref[cs][student_id])),name=f"02_course_{cs}_constr")
            
            const_vars[f"03_course_{cs}_sections_waitlist_constr"] = solver.Add(solver.Sum(course_a_ref[cs][student_id]) <= 1, name=f"03_course_{cs}_sections_waitlist_constr")
            const_vars[f"03_course_{cs}_sections_constr"] = solver.Add(solver.Sum(course_b_ref[cs][student_id]) <= 1, name=f"03_course_{cs}_sections_constr")
            
            const_vars[f"04_one_regstat_{student_id}"] = solver.Add(solver.Sum(course_a_ref[cs][student_id]) + solver.Sum(course_b_ref[cs][student_id]) == 1, name=f"04_one_regstat_{student_id}")
            
    # Minimization function 
    # -> For later * a_decision_vars[student_id]
    solver.Minimize(solver.Sum([w_decision_vars[student_id]
                                for student_id in range(1,reg_term_values[a_sem])
                               if b_decision_vars[student_id] == 1]))
    solver.Solve()
    break

In [333]:
solver.NumVariables()

2744

In [334]:
solver.NumConstraints()

4119

In [112]:
print('Objective value =', solver.Objective().Value())

Objective value = 96.0


In [116]:
list(x_decision_vars.values())[6].name()

'x(EC-MS-DAEN,DAEN500,DL2,7,202010)'

In [120]:
for i in list(x_decision_vars.values()):
    print(i.SolutionValue())

1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


**NOW WE CAN GET INTO THE INTERESTING STUFF**

This is for when I have more data; only focusing on what I know for certain given the total number of students available.

In [ ]:
waitlist_courses = total_courses.loc[total_courses["reg_status"] == "W",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
dropped_courses = total_courses.loc[total_courses["reg_status"] == "D",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

In [ ]:
waitlist_courses = waitlist_courses.rename(columns={'reg_status':'waitlist'})
required_courses = required_courses.rename(columns={'reg_status':'registered'})
dropped_courses = dropped_courses.rename(columns={'reg_status':'dropped'})

In [ ]:
total_courses = required_courses.merge(waitlist_courses, on=["reg_term_code", "stu_prog","crs","sect_id"], how="left").fillna(0)
total_courses = total_courses.merge(dropped_courses, on=["reg_term_code", "stu_prog","crs","sect_id"], how="left").fillna(0)